In [1]:
import pandas as pd
import numpy as np
import cv2
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.neighbors import NearestNeighbors
import os
from PIL import Image
import numpy as np
import pywt
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

In [2]:
train_info = pd.read_csv("Data/Train.csv")
train_info['ClassId'].value_counts()
# train_info

#test_info = pd.read_csv("Data/Test.csv")
#test_info['ClassId'].value_counts() 

,Width,Height,Roi.X1,Roi.Y1,Roi.X2,Roi.Y2,ClassId,Path
0,29,30,5,6,24,25,0,Train/0/00000_00000_00000.png
1,30,30,5,5,25,25,0,Train/0/00000_00000_00001.png
2,30,30,5,5,25,25,0,Train/0/00000_00000_00002.png
3,31,31,5,5,26,26,0,Train/0/00000_00000_00003.png
4,30,32,5,6,25,26,0,Train/0/00000_00000_00004.png
...,...,...,...,...,...,...,...,...
12196,47,47,5,5,42,42,9,Train/9/00009_00048_00022.png
12197,51,51,6,6,45,46,9,Train/9/00009_00048_00023.png
12198,53,53,6,5,48,48,9,Train/9/00009_00048_00024.png
12199,57,57,6,5,51,51,9,Train/9/00009_00048_00025.png


In [12]:
def resize_and_save_img(src, destination):
    original_img = cv2.imread(src)
    old_image_height, old_image_width, channels = original_img.shape
    new_image_width = 60        
    new_image_height = 60
    color = (255,255,255)

    result = np.full((new_image_height, new_image_width, channels), color, dtype=np.uint8)
    # compute center offset
    x_center = (new_image_width - old_image_width) // 2
    y_center = (new_image_height - old_image_height) // 2

    # copy img image into center of result image
    result[y_center:y_center+old_image_height, x_center:x_center+old_image_width] = original_img

    Image.fromarray(result).save(destination)

resize_and_save_img("Data/Test/00001.png", "NewData/00001.png")